# ゼロから作る Deep Learning 4 強化学習編 勉強ノート 第6章 〜TD 法〜

ここでは、環境のモデルを使わずに、さらには行動を1つ行うたびに価値関数を更新する「<font color="red">**TD 法**</font>」について扱う。

## TD 法による方策評価

TD 法は、これまでに学んだ「モンテカルロ法」と「動的計画法」を合わせたような手法である。  
モンテカルロ法のようにエピソードの終わりを待つのではなく、一定の時間が進むごとに方策の評価と改善を行う。

### TD 法の導出

収益は次のように定義された。

\begin{eqnarray*}
G_t &=& R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + \cdots \\
&=& R_t + \gamma G_{t+1}
\end{eqnarray*}

この収益を使って、価値関数は次のように期待値として定義された。

\begin{eqnarray*}
v_\pi(s) &=& \mathbb{E}_\pi[G_t|S_t = s] \\
&=& \mathbb{E}_\pi[R_t + \gamma G_{t+1}|S_t = s]
\end{eqnarray*}

モンテカルロ法では期待値を計算する代わりに、実際に得られた収益のサンプルデータを平均することで近似する。  
平均には、標本平均と指数移動平均の2つがある。  
指数移動平均を実現するには、新しい収益が得られるたびに固定値 $\alpha$ で更新する。

$$V'_\pi(S_t) = V_\pi(S_t) + \alpha \{G_t - V_\pi(S_t)\}$$

動的計画法では、数式通りに期待値を計算する。

\begin{eqnarray*}
v_\pi(s) &=& \mathbb{E}_\pi[R_t + \gamma G_{t+1}|S_t = s] \\
&=& \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma v_{\pi}(s') \right\}
\end{eqnarray*}

状態遷移確率 $p(s'|s, a)$ と報酬関数 $r(s, a, s')$ を使って期待値を計算する。  
これはベルマン方程式であり、 DP はこれに基づいて価値関数を逐次更新する。

$$V'_\pi(s) = \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma V_\pi(s') \right\}$$

DP では、今の価値関数の推定値を次の価値関数の推定値を使って更新する。  
この原理は「ブートストラップ」と呼ばれた。  
一方、モンテカルロ法は実際に得られた経験によって今の価値関数を更新する。  
この2つの手法を融合させたのが TD 法である。  
重要な点は以下の2つである。

- DP のようにブートストラップにより価値関数を逐次更新できる
- モンテカルロ法のように環境に関する知識を必要とせずにサンプリングされたデータを使って価値関数を更新できる

\begin{eqnarray*}
v_\pi(s) &=& \sum_{a, s'} \pi(a|s) p(s'|s, a) \left\{ r(s, a, s') + \gamma v_{\pi}(s') \right\} \\
&=& \mathbb{E}_\pi[R_t + \gamma v_\pi(S_{t+1})|S_t = s]
\end{eqnarray*}

TD 法ではこれを用いて価値関数を更新するにあたり、 $R_t + \gamma v_\pi(S_{t+1})$ の部分をサンプルデータから近似する。

$$V'_\pi(S_t) = V_\pi(S_t) + \alpha \{ R_t + \gamma V_\pi(S_{t+1}) - V_\pi(S_t) \}$$

$V_\pi$ は価値関数の推定値であり、目的地(ターゲット)が $R_t + \gamma V_\pi(S_{t+1})$ になる。  
この目的地は <font color="red">**TD ターゲット**</font>と呼ばれる。

### モンテカルロ法と TD 法の比較

モンテカルロ法と TD 法の式を比較する。

\begin{eqnarray*}
\text{【モンテカルロ法】} V'_\pi(S_t) &=& V_\pi(S_t) + \alpha \{G_t - V_\pi(S_t)\} \\
\text{【TD 法】} V'_\pi(S_t) &=& V_\pi(S_t) + \alpha \{ R_t + \gamma V_\pi(S_{t+1}) - V_\pi(S_t) \}
\end{eqnarray*}

モンテカルロ法では $G_t$ をターゲットとし、その方向へと $V_\pi$ を更新する。  
$G_t$ はゴールに辿り着いてから得られる収益のサンプルデータである。  
一方、 TD 法のターゲットは、1ステップ先の情報をもとに計算する。  
時間が1ステップ進むごとに価値関数を更新できるため、 TD 法の方が効率の良い学習が期待できる。  
また、モンテカルロ法では分散が大きくなってしまうのに対し、 TD 法は1ステップ先のデータに基づくので、その変動は小さくなる。

### TD 法の実装(*コーディング*)

ランダムな方策を持つエージェントに対して TD 法を使って方策を評価する。

In [ ]:
# ライブラリのインポート
from collections import defaultdict

import numpy as np

In [ ]:
# GridWorld クラスの実装
class GridWorld:
    def __init__(self):
        self.action_space = [0, 1, 2, 3]
        self.action_meaning = {
            0: "UP",
            1: "DOWN",
            2: "LEFT",
            3: "RIGHT"
        }
        self.reward_map = np.array(
            [
                [0, 0, 0, 1.0],
                [0, None, 0, -1.0],
                [0, 0, 0, 0]
            ]
        )
        self.goal_state = (0, 3)
        self.wall_state = (1, 1)
        self.start_state = (2, 0)
        self.agent_state = self.start_state

    @property
    def height(self):
        return len(self.reward_map)

    @property
    def width(self):
        return len(self.reward_map[0])

    @property
    def shape(self):
        return self.reward_map.shape

    def actions(self):
        return self.action_space

    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h, w)

    def next_state(self, state, action):
        action_move_map = [
            (-1, 0), (1, 0), (0, -1), (0, 1)
        ]
        move = action_move_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        ny, nx = next_state

        if nx < 0 or nx >= self.width or ny < 0 or ny >= self.height:
            next_state = state
        elif next_state == self.wall_state:
            next_state = state

        return next_state

    def reward(self, state, action, next_state):
        return self.reward_map[next_state]

    def step(self, action):
        state = self.agent_state
        next_state = self.next_state(state, action)
        reward = self.reward(state, action, next_state)
        done = (next_state == self.goal_state)

        self.agent_state = next_state

        return next_state, reward, done

    def reset(self):
        self.agent_state = self.start_state
        return self.agent_state

In [ ]:
# TdAgent クラスの実装
class TdAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.01
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.V = defaultdict(lambda: 0)

    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def eval(self, state, reward, next_state, done):
        next_V = 0 if done else self.V[next_state]
        target = reward + self.gamma * next_V

        self.V[state] += (target - self.V[state]) * self.alpha

エージェントを実際に行動させて方策評価を行う。

In [ ]:
# 各クラスのインスタンスを生成
env = GridWorld()
agent = TdAgent()

# 1000回のエピソードで実行
episodes = 1000
for episode in range(episodes):
    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.eval(state, reward, next_state, done)
        if done:
            break
        state = next_state

print(agent.V)

ランダムな方策を持つエージェントの価値関数を評価することが出来た。

## SARSA

前節では TD 法による方策評価を行った。  
方策評価が終われば、次は方策制御である。  
ここでは「方策オン型」の SARSA という手法に取り組む。

### 方策オン型の SARSA

方策制御を行う場合には、 $V_\pi(s)$ ではなく $Q_\pi(s, a)$ を対象にしなければいけない。  
改善フェーズでは方策を greedy 化する必要があり、 $V_\pi(s)$ の場合は環境のモデルが必要になる。  
一方、 $Q_\pi(s, a)$ であれば

$$
\newcommand{\argmax}{\mathop{\rm arg~max}\limits} \\
\mu(s) = \argmax_a Q_\pi(s, a)
$$

と計算でき、環境のモデルを必要としない。

それでは、状態価値関数 $V_\pi(S_t)$ から行動価値関数 $Q_\pi(S_t, A_t)$ へと TD 法を変更する。

$$Q'_\pi(S_t, A_t) = Q_\pi(S_t, A_t) + \alpha \{ R_t + \gamma Q_\pi(S_{t+1}, A_{t+1}) - Q_\pi(S_t, A_t) \}$$

これが Q 関数を対象にした TD 法の更新式である。  
$Q_\pi(S_t, A_t)$ の更新が終わればすぐに改善フェーズに進める。  
この例では、 $Q_\pi(S_t, A_t)$ が更新されるので、状態 $S_t$ における方策が変更される可能性がある。

\begin{equation}
\newcommand{\argmax}{\mathop{\rm arg~max}\limits} \\
\pi'(a|S_t) = \left\{ \,
    \begin{aligned}
    \argmax_a Q_\pi(S_t, a)&\quad \text{(1 - ε の確率)} \\
    \text{ランダムな行動}&\quad \text{(ε の確率)}
    \end{aligned}
\right.
\end{equation}

ε の確率でランダムな行動を選び、それ以外は greedy な行動を選ぶ。  
greedy な行動によって方策は改善され、ランダムな行動によって探索が行われる。  
この ε-greedy 法によって状態 $S_t$ における行動の選び方が更新される。

### SARSA の実装(*コーディング*)

`SarsaAgent` クラスを実装する。

In [ ]:
# ライブラリのインポート
from collections import deque

In [ ]:
# greedy_probs 関数を定義
def greedy_probs(
        Q: dict, state: tuple[int, int], eps: float=0, action_size: int=4
    ) -> dict:
    qs = [Q[(state, action)] for action in range(action_size)]
    max_action = np.argmax(qs)

    base_prob = eps / action_size
    action_probs = {action: base_prob for action in range(action_size)}
    action_probs[max_action] += (1 - eps)

    return action_probs

In [ ]:
# SarsaAgent クラスの実装
class SarsaAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.eps = 0.1
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.memory = deque(maxlen=2)

    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def reset(self):
        self.memory.clear()

    def update(self, state, action, reward, done):
        self.memory.append((state, action, reward, done))
        if len(self.memory) < 2:
            return

        state, action, reward, done = self.memory[0]
        next_state, next_action, _, _ = self.memory[1]

        next_q = 0 if done else self.Q[next_state, next_action]

        target = reward + self.gamma * next_q
        self.Q[state, action] += (target - self.Q[state, action]) * self.alpha

        self.pi[state] = greedy_probs(self.Q, state, self.eps)

この `SarsaAgent` クラスを動かしてみる。

In [ ]:
# 各クラスのインスタンスを生成
env = GridWorld()
agent = SarsaAgent()

# 10000回のエピソードで実行
episodes = 10000
for episode in range(episodes):
    state = env.reset()
    agent.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, done)

        if done:
            agent.update(next_state, None, None, None)
            break

        state = next_state

print(agent.Q)

方策にはランダム性があるため、できるだけ負の報酬から遠ざかるような動きが見られる。

## 方策オフ型の SARSA

まずは方策オフ型の SARSA を導出し、その後 Q 学習に進む。

### 方策オフ型と重点サンプリング

方策オフ型では、エージェントは挙動方策とターゲット方策の2つの方策を持つ。  
挙動方策で多様な行動を行ってサンプルデータを集め、それを使ってターゲット方策を greedy に更新する。  
$Q_\pi(S_t, A_t)$ を更新する場合を考え、方策 $\pi$ によって行動が選ばれることを明示すると SARSA の更新式は次のようになる。

$$
\text{sampling: } A_{t+1} \sim \pi \\
Q'_\pi(S_t, A_t) = Q_\pi(S_t, A_t) + \alpha \{ R_t + \gamma Q_\pi(S_{t+1}, A_{t+1}) - Q_\pi(S_t, A_t) \}
$$

$Q_\pi(S_t, A_t)$ を $R_t + \gamma Q_\pi(S_{t+1}, A_{t+1})$ の方向へと更新することを表している。  
この $R_t + \gamma Q_\pi(S_{t+1}, A_{t+1})$ は「TD ターゲット」と呼ばれる。  
行動 $A_{t+1}$ が方策 $b$ によってサンプリングされた場合を考える。  
重み $\rho$ によって TD ターゲットを補正し、重点サンプリングを行う。

$$\rho = \frac{\pi(A_{t+1}|S_{t+1})}{b(A_{t+1}|S_{t+1})}$$

よって、方策オフ型の SARSA の更新式は次のようになる。

$$
\text{sampling: } A_{t+1} \sim b \\
Q'_\pi(S_t, A_t) = Q_\pi(S_t, A_t) + \alpha \{ \rho \left( R_t + \gamma Q_\pi(S_{t+1}, A_{t+1}) \right) - Q_\pi(S_t, A_t) \}
$$

### 方策オフ型の SARSA の実装(*コーディング*)

方策オフ型の SARSA を実装する。

In [ ]:
# SarsaOffPolicyAgent クラスの実装
class SarsaOffPolicyAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.eps = 0.1
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.b = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.memory = deque(maxlen=2)

    def get_action(self, state):
        action_probs = self.b[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def reset(self):
        self.memory.clear()

    def update(self, state, action, reward, done):
        self.memory.append((state, action, reward, done))
        if len(self.memory) < 2:
            return

        state, action, reward, done = self.memory[0]
        next_state, next_action, _, _ = self.memory[1]

        if done:
            next_q = 0
            rho = 1
        else:
            next_q = self.Q[next_state, next_action]
            rho = self.pi[next_state][next_action] / \
                self.b[next_state][next_action]

        target = rho * (reward + self.gamma * next_q)
        self.Q[state, action] += (target - self.Q[state, action]) * self.alpha

        self.pi[state] = greedy_probs(self.Q, state, 0)
        self.b[state] = greedy_probs(self.Q, state, self.eps)

ここで実装した `SarsaOffPolicyAgent` クラスを使ってグリッドワールドの問題を解かせる。

In [ ]:
# 各クラスのインスタンスを生成
env = GridWorld()
agent = SarsaOffPolicyAgent()

# 10000回のエピソードで実行
episodes = 10000
for episode in range(episodes):
    state = env.reset()
    agent.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, done)

        if done:
            agent.update(next_state, None, None, None)
            break

        state = next_state

print(agent.Q)

## Q 学習

重点サンプリングは結果が、特に2つの方策の確率分布が異なれば異なるほど不安定になりやすいという問題があるため、できることならば避けたい手法である。  
方策オフ型の SARSA ではこれが必要であり、更新式にあるターゲットも大きく変動するため Q 関数の更新が不安定になる。  
<font color="red">**Q 学習**</font>(Q-learning)である。  
Q 学習は次の3つの特徴を持つ。

- TD 法
- 方策オフ型
- 重点サンプリングを行わない

### ベルマン方程式と SARSA

まずは SARSA とベルマン方程式の関係性から確認する。  
方策 $\pi$ における Q 関数を $q_\pi(s, a)$ としたとき、ベルマン方程式は次の式で表される。

$$q_\pi(s, a) = \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma \sum_{a'} \pi (a'|s') q_{\pi}(s', a') \right\}$$

ベルマン方程式で重要なのは次の2点である。

- 環境の状態遷移確率 $p(s'|s, a)$ によって「次の全ての状態遷移」を考慮していること
- エージェントの方策 $\pi$ によって「次の全ての行動」を考慮していること

SARSA はベルマン方程式の「サンプリング版」と見做すことができる。  
SARSA では次の状態 $S_{t+1}$ は $p(s'|s, a)$ に基づきサンプリングする。  
そして、次の行動 $A_{t+1}$ は方策 $\pi(a|s)$ に基づきサンプリングする。  
このとき、 SARSA の TD ターゲットは $R_t + \gamma Q_\pi(S_{t+1}, A_{t+1})$ になる。  
このターゲットの方向に Q 関数を少しだけ更新する。

### ベルマン最適方程式と Q 学習

価値反復法は、最適方策を得るための「評価」と「改善」という2つのプロセスを1つにまとめた手法である。  
価値反復法の重要な点は、ベルマン最適方程式に基づくただ1つの更新式を繰り返すことで、最適方策が得られることである。  
ここではベルマン最適方程式による更新で、なおかつそれを「サンプリング版」にした手法について考える。  
初めに、 Q 関数のベルマン最適方程式を見てみる。

$$q_*(s, a) = \sum_{s'} p(s'|s, a) \left\{ r(s, a, s') + \gamma \max_{a'} q_*(s', a') \right\}$$

これを「サンプリング版」に書き換える。  
Q 学習では、推定値 $Q(S_t, A_t)$ のターゲットは $R_t + \gamma \max_a Q(S_{t+1}, a)$ になる。  
このターゲットの方向へと Q 関数を更新する。

$$Q'(S_t, A_t) = Q(S_t, A_t) + \alpha \left\{ R_t + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right\}$$

この式に基づき Q 関数を繰り返し更新することで最適方策における Q 関数へと近づく。

### Q 学習の実装(*コーディング*)

Q 学習を行う `QLearningAgent` クラスを実装する。

In [ ]:
# QLearningAgent クラスの実装
class QLearningAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.eps = 0.1
        self.action_size = 4

        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)
        self.b = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)

    def get_action(self, state):
        action_probs = self.b[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def update(self, state, action, reward, next_state, done):
        if done:
            next_q_max = 0
        else:
            next_qs = [self.Q[next_state, a] for a in range(self.action_size)]
            next_q_max = max(next_qs)

        target = reward + self.gamma * next_q_max
        self.Q[state, action] += (target - self.Q[state, action]) * self.alpha

        self.pi[state] = greedy_probs(self.Q, state, eps=0)
        self.b[state] = greedy_probs(self.Q, state, self.eps)

これを動かしてみる。

In [ ]:
# 各クラスのインスタンスを生成
env = GridWorld()
agent = QLearningAgent()

# 10000回のエピソードで実行
episodes = 10000
for episode in range(episodes):
    state = env.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.update(state, action, reward, next_state, done)
        if done:
            break

        state = next_state

print(agent.Q)

結果は毎回変わるが、多くの場合最適方策が得られることがわかる。

## 分布モデルとサンプルモデル

本章でこれまで行ってきた実装は分布モデルであり、サンプルモデルの方がよりシンプルに実装できることを示す。

### 分布モデルとサンプルモデル(*コーディング*)

分布モデルとは、確率分布を明示的に保持するモデルである。  
例えば、ランダムに行動するエージェントを分布モデルとして実装すると次のようになる。

In [ ]:
# 分布モデルとして実装した RandomAgent クラス
class RandomAgent:
    def __init__(self):
        random_actions = {
            0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25
        }
        self.pi = defaultdict(lambda: random_actions)

    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

一方で、サンプルモデルはサンプリングできることだけが条件のモデルであり、確率分布を持つ必要がないため、分布モデルよりシンプルに実装できる。

In [ ]:
# サンプルモデルとして実装した RandomAgent クラス
class RandomAgent:
    def get_action(self, state):
        return np.random.choice(4)

ここでは確率分布を持たずに、単に4つの行動からランダムに1つを選びだすように実装する。

### サンプルモデル版の Q 学習

前節で Q 学習のエージェントを分布モデルとして実装した。  
今回はサンプルモデルで実装する。

In [ ]:
# サンプルモデルとして実装した QLearningAgent クラス
class QLearningAgent:
    def __init__(self):
        self.gamma = 0.9
        self.alpha = 0.8
        self.eps = 0.1
        self.action_size = 4
        self.Q = defaultdict(lambda: 0)

    def get_action(self, state):
        if np.random.rand() < self.eps:
            return np.random.choice(self.action_size)
        else:
            qs = [self.Q[state, a] for a in range(self.action_size)]
            return np.argmax(qs)

    def update(self, state, action, reward, next_state, done):
        if done:
            next_q_max = 0
        else:
            next_qs = [self.Q[next_state, a] for a in range(self.action_size)]
            next_q_max = max(next_qs)

        target = reward + self.gamma * next_q_max
        self.Q[state, action] += (target - self.Q[state, action]) * self.alpha

見てわかる通り、このコードでは方策を確率分布として保持していない。  
確率分布を保持する必要がないのでよりシンプルな実装ができている。